In [1]:
import pandas as pd 


In [ ]:
import pandas as pd
import requests
import os

def fetch_google_metadata(isbn):
    """Direct API Flow: Purged of Description to eliminate murky data."""
    # Systemic Fix: Removes internal spaces (8 1 7... -> 817...)
    isbn_clean = str(isbn).replace(" ", "").split('.')[0].strip()
    if not isbn_clean or isbn_clean.lower() in ['nan', 'none']:
        return None
        
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn_clean}"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if "items" in data:
                info = data["items"][0]["volumeInfo"]
                return {
                    "Title": str(info.get("title", "N/A")),
                    "Author": ", ".join(info.get("authors", ["Unknown"])),
                    "Publishers": str(info.get("publisher", "N/A")),
                    "PublisherDate": str(info.get("publishedDate", "N/A")),
                    "Genre": ", ".join(info.get("categories", ["General"]))
                }
    except Exception: pass
    return None

def process_and_convert(input_file, output_csv):
    # 1. Structural Layer: XLSX to CSV Conversion
    if not os.path.exists(input_file):
        print(f"Error: {input_file} not found.")
        return

    # Kills the UnicodeDecodeError by reading binary first
    if input_file.endswith('.xlsx'):
        print(f"Detecting Binary Excel Layer. Converting {input_file} to CSV...")
        df = pd.read_excel(input_file)
    else:
        df = pd.read_csv(input_file)

    original_df = df.copy() 
    total_rows = len(df)
    
    # 2. Logic Orange: Identify headers and initialize types
    search_col = next((c for c in ['Isbn', 'isbn13', 'ISBN', 'isbn'] if c in df.columns), None)
    title_col = next((c for c in ['Title', 'title'] if c in df.columns), "Title")
    
    if not search_col:
        print("Critical Error: No ISBN column found.")
        return

    # Force columns to String type (Description removed from this list)
    target_cols = ['Title', 'Author', 'Publishers', 'PublisherDate', 'Genre']
    for col in target_cols:
        if col not in df.columns: df[col] = ""
        df[col] = df[col].astype(str)

    # 3. Active Flow: Synchronized Mapping
    print(f"Total rows: {total_rows}. Extracting Genre for suggestions...")

    for index, row in df.iterrows():
        isbn = row[search_col]
        if pd.isna(isbn):
            continue

        metadata = fetch_google_metadata(isbn)
        
        if metadata:
            for key, val in metadata.items():
                df.at[index, key] = val
        
        # Incremental Save every 50 rows
        if (index + 1) % 50 == 0:
            df.to_csv(output_csv, index=False)
            print(f"--- Progress: {((index+1)/total_rows)*100:.2f}% ---", end="\r")

    # 4. Final Save and Audit
    df.to_csv(output_csv, index=False)
    print(f"\nGrid Enriched. System saved to {output_csv}")
    
    # --- Final Integrity Audit ---
    mismatches = []
    for idx in range(total_rows):
        orig_t = str(original_df.at[idx, title_col]).lower().strip()
        new_t = str(df.at[idx, 'Title']).lower().strip()
        if orig_t != new_t and new_t not in ["nan", "", "n/a"]:
            mismatches.append(idx)

    print("\n" + "="*40)
    print("--- INTEGRATED GENESIS FINAL REPORT ---")
    print(f"Total Rows:           {total_rows}")
    print(f"Title Discrepancies:  {len(mismatches)}")
    print(f"Features Extracted:   Genre")
    print("="*40)

# --- EXECUTION ---
process_and_convert("Books_Data_for_Student.xlsx", "library_enriched.csv")

Detecting Binary Excel Layer. Converting Books_Data_for_Student.xlsx to CSV...
Total rows: 50623. Extracting Genre for suggestions...
